# Custom RAG System (Gemini Flash)

## Phase 1: Environment & Project Setup

This phase ensures:
- Clean project structure
- Isolated Python environment
- Secure secrets management
- Reproducible execution

In [ ]:
import sys
print("Python executable:", sys.executable)
print("Python version:", sys.version)

In [ ]:
import platform
print("OS:", platform.system())

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

print("GOOGLE_API_KEY loaded:", bool(os.getenv("GOOGLE_API_KEY")))
print("PINECONE_API_KEY loaded:", bool(os.getenv("PINECONE_API_KEY")))
print("PINECONE_ENV loaded:", bool(os.getenv("PINECONE_ENV")))

In [ ]:
print("Phase 1 setup complete and verified.")

# Phase 2: Document Ingestion & Text Extraction

This phase:
- Loads PDFs from disk
- Extracts page-level text
- Preserves metadata (source + page)
- Performs validation and sanity checks

In [ ]:
from pathlib import Path
from langchain.document_loaders import PyPDFLoader

In [ ]:
PDF_DIR = Path("data/pdfs")
assert PDF_DIR.exists(), "data/pdfs folder does not exist"

pdf_files = sorted(PDF_DIR.glob("*.pdf"))

print(f"Found {len(pdf_files)} PDF files:")
for f in pdf_files:
    print("-", f.name)

assert len(pdf_files) > 0, "No PDFs found. Add files to data/pdfs/"

In [ ]:
documents = []

for pdf_path in pdf_files:
    print(f"\nLoading: {pdf_path.name}")
    loader = PyPDFLoader(str(pdf_path))
    pages = loader.load()

    print(f"  Pages extracted: {len(pages)}")
    documents.extend(pages)

print(f"\nTotal pages loaded from all PDFs: {len(documents)}")

In [ ]:
sample = documents[0]
print("Metadata:", sample.metadata)
print("\n--- Content Preview (first 1000 chars) ---\n")
print(sample.page_content[:1000])

In [ ]:
empty_pages = [i for i, d in enumerate(documents) if len(d.page_content.strip()) < 50]

print("Total pages:", len(documents))
print("Empty or near-empty pages:", len(empty_pages))

if len(empty_pages) > 0:
    print("Indices of empty pages:", empty_pages[:10])

In [ ]:
total_chars = sum(len(d.page_content) for d in documents)
avg_chars = total_chars / len(documents)

print("Total characters:", total_chars)
print("Average characters per page:", int(avg_chars))

In [ ]:
print("Phase 2 complete: Documents ingested and validated.")

# Phase 3: Text Normalization & Chunking

This phase:
- Normalizes raw extracted text
- Removes formatting noise
- Prepares semantically meaningful chunks
- Preserves source metadata for traceability

In [ ]:
import re
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
def normalize_text(text: str) -> str:
    # Collapse multiple newlines
    text = re.sub(r"\n+", "\n", text)

    # Collapse excessive whitespace
    text = re.sub(r"\s+", " ", text)

    # Strip edges
    text = text.strip()

    return text

In [ ]:
for doc in documents:
    doc.page_content = normalize_text(doc.page_content)

print("Text normalization complete.")

In [ ]:
print("=== NORMALIZED TEXT SAMPLE ===\n")
print(documents[0].page_content[:1000])

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=150,
    separators=["\n\n", "\n", ". ", " ", ""]
)

In [ ]:
chunks = text_splitter.split_documents(documents)

print("Total pages:", len(documents))
print("Total chunks created:", len(chunks))

In [ ]:
sample_chunk = chunks[0]

print("Chunk metadata:", sample_chunk.metadata)
print("\n--- Chunk preview ---\n")
print(sample_chunk.page_content[:1000])

In [ ]:
sizes = [len(c.page_content) for c in chunks]

print("Min chunk size:", min(sizes))
print("Max chunk size:", max(sizes))
print("Avg chunk size:", sum(sizes) // len(sizes))

In [ ]:
print("Phase 3 complete: Text normalized and chunked successfully.")

# Phase 4: Embedding Generation (Sentence Transformers)

This phase:
- Initializes a HuggingFace embedding model
- Converts text chunks into vector embeddings
- Verifies embedding shape and consistency

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np

In [ ]:
EMBEDDING_MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"

print("Loading embedding model:", EMBEDDING_MODEL_NAME)
embedder = SentenceTransformer(EMBEDDING_MODEL_NAME)

print("Model loaded.")

In [ ]:
texts = [chunk.page_content for chunk in chunks]
print("Total chunks to embed:", len(texts))

In [ ]:
embeddings = embedder.encode(
    texts,
    batch_size=32,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True
)

print("Embeddings generated.")

In [ ]:
print("Embeddings array shape:", embeddings.shape)
print("Single embedding dimension:", embeddings.shape[1])

In [ ]:
print("Sample embedding (first 10 values):")
print(embeddings[0][:10])
print("Vector norm:", np.linalg.norm(embeddings[0]))

In [ ]:
assert embeddings.shape[0] == len(chunks), "Mismatch between chunks and embeddings!"
assert embeddings.shape[1] == 384, "Unexpected embedding dimension!"

print("Embedding consistency checks passed.")

In [ ]:
print("Phase 4 complete: Embeddings generated and validated.")

# Phase 5: Vector Database (Pinecone Serverless)

This phase:
- Initializes Pinecone Serverless client
- Creates a new serverless index (if needed)
- Batches and upserts vectors with metadata

In [ ]:
import os
from pinecone import Pinecone, ServerlessSpec
from dotenv import load_dotenv

load_dotenv()

In [ ]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
assert PINECONE_API_KEY is not None, "Missing PINECONE_API_KEY"

pc = Pinecone(api_key=PINECONE_API_KEY)
print("Pinecone client initialized.")

In [ ]:
INDEX_NAME = "notebooklm-rag-antigravity"

existing_indexes = [i["name"] for i in pc.list_indexes()]

if INDEX_NAME not in existing_indexes:
    print("Creating new serverless index:", INDEX_NAME)
    pc.create_index(
        name=INDEX_NAME,
        dimension=embeddings.shape[1],
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )
else:
    print("Using existing index:", INDEX_NAME)

In [ ]:
index = pc.Index(INDEX_NAME)

In [ ]:
vectors = []

for i, (chunk, vector) in enumerate(zip(chunks, embeddings)):
    vectors.append({
        "id": f"chunk-{i}",
        "values": vector.tolist(),
        "metadata": {
            "source": chunk.metadata.get("source", ""),
            "page": chunk.metadata.get("page", ""),
            "text": chunk.page_content[:1000]  # metadata size safety
        }
    })

print("Prepared vectors:", len(vectors))

In [ ]:
BATCH_SIZE = 100

for i in range(0, len(vectors), BATCH_SIZE):
    batch = vectors[i:i+BATCH_SIZE]
    index.upsert(vectors=batch)
    print(f"Uploaded {min(i + BATCH_SIZE, len(vectors))} / {len(vectors)} vectors")

In [ ]:
stats = index.describe_index_stats()
print("Index stats:", stats)

In [ ]:
print("Phase 5 complete: Vectors successfully stored in Pinecone Serverless.")

# Phase 6: Semantic Retrieval Layer

This phase:
- Encodes user queries
- Performs similarity search in Pinecone
- Retrieves top-k relevant chunks
- Allows inspection of retrieved context

In [ ]:
import numpy as np

In [ ]:
def embed_query(query: str) -> np.ndarray:
    vec = embedder.encode(
        [query],
        convert_to_numpy=True,
        normalize_embeddings=True
    )
    return vec[0]

In [ ]:
def retrieve_top_k(query: str, k: int = 5):
    query_vec = embed_query(query)

    result = index.query(
        vector=query_vec.tolist(),
        top_k=k,
        include_metadata=True
    )

    return result["matches"]

In [ ]:
TEST_QUERY = "What is the main topic discussed in these documents?"

results = retrieve_top_k(TEST_QUERY, k=5)

print("Retrieved chunks:", len(results))

In [ ]:
for i, match in enumerate(results, 1):
    print(f"\n--- Result {i} ---")
    print("Score:", match["score"])
    print("Source:", match["metadata"].get("source"))
    print("Page:", match["metadata"].get("page"))
    print("\nText Preview:\n")
    print(match["metadata"].get("text")[:1000])

In [ ]:
assert len(results) > 0, "No retrieval results returned!"
print("Retrieval sanity check passed.")

In [ ]:
print("Phase 6 complete: Semantic retrieval layer operational.")

# Phase 7: RAG Answer Generation (Gemini Flash)

This phase:
- Integrates Gemini Flash as the LLM
- Combines semantic retrieval + generation
- Uses strict grounding to prevent hallucination
- Produces user-facing answers

In [ ]:
import os
import google.generativeai as genai
from dotenv import load_dotenv

load_dotenv()

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
assert GOOGLE_API_KEY is not None, "Missing GOOGLE_API_KEY"

genai.configure(api_key=GOOGLE_API_KEY)

model = genai.GenerativeModel("gemini-1.5-flash")
print("Gemini Flash initialized.")

In [ ]:
def build_context(retrieved_matches, max_chars=6000):
    context_blocks = []
    total_chars = 0

    for match in retrieved_matches:
        text = match["metadata"].get("text", "")
        source = match["metadata"].get("source", "")
        page = match["metadata"].get("page", "")

        block = f"[Source: {source}, Page: {page}]\n{text}\n"
        
        if total_chars + len(block) > max_chars:
            break

        context_blocks.append(block)
        total_chars += len(block)

    return "\n---\n".join(context_blocks)

In [ ]:
def build_rag_prompt(context: str, question: str) -> str:
    prompt = f"""
You are a factual assistant.

Answer the question strictly using ONLY the context below.
If the answer is not present in the context, say:
"I don't have enough information in the provided documents."

CONTEXT:
{context}

QUESTION:
{question}

ANSWER:
"""
    return prompt.strip()

In [ ]:
def rag_answer(query: str, k: int = 5):
    # Step 1: Retrieve relevant chunks
    retrieved = retrieve_top_k(query, k=k)

    # Step 2: Build context window
    context = build_context(retrieved)

    # Step 3: Build grounded prompt
    prompt = build_rag_prompt(context, query)

    # Step 4: Generate answer using Gemini
    response = model.generate_content(prompt)

    return {
        "query": query,
        "answer": response.text,
        "context": context,
        "retrieved": retrieved
    }

In [ ]:
TEST_QUESTION = "Summarize the main topics discussed in these documents."

result = rag_answer(TEST_QUESTION, k=5)

print("QUESTION:\n", result["query"])
print("\nANSWER:\n", result["answer"])

In [ ]:
print("\n=== CONTEXT USED FOR ANSWER ===\n")
print(result["context"])

In [ ]:
print("\n🔎 Interactive Q&A Mode (type 'exit' to quit)\n")

while True:
    user_question = input("Ask a question: ").strip()

    if user_question.lower() in ["exit", "quit"]:
        print("Exiting Q&A.")
        break

    if len(user_question) < 3:
        print("Please ask a more specific question.")
        continue

    print("\n🔍 Searching documents and generating answer...\n")

    result = rag_answer(user_question, k=5)

    print("❓ QUESTION:")
    print(user_question)

    print("\n✅ ANSWER:")
    print(result["answer"])

    print("\n📚 SOURCES USED:")
    for i, match in enumerate(result["retrieved"], 1):
        meta = match["metadata"]
        print(f"{i}. {meta.get('source')} (page {meta.get('page')})")

    print("\n" + "="*80 + "\n")

In [ ]:
print("Phase 7 complete: End-to-end RAG system operational with Gemini Flash.")